In [ ]:
from transformers import BertTokenizer, TFBertModel

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = TFBertModel.from_pretrained("bert-base-multilingual-cased")
text = "Replace me by any text you'd like."
encoded_input = tokenizer(text, return_tensors='tf')
output = model(encoded_input)

In [1]:
import pandas as pd
import numpy as np
from nltk.translate import bleu
import tensorflow as tf

d = pd.read_csv("apdf.csv")

In [ ]:
d.head()

In [ ]:
testdata = d.head(1)

In [ ]:
testdata["en_text"][0]

In [ ]:
testlist = ["sentence 1",  "sentence 2"]

In [ ]:
encoded_input = tokenizer(testdata["en_text"][0], return_tensors='tf')
output = model(encoded_input)

In [ ]:
tokenizer(testdata["en_text"][0])["input_ids"]

In [ ]:
tokenizer.decode(tokenizer(testdata["en_text"][0])["input_ids"])

In [ ]:
from transformers import BertModel, BertConfig

In [ ]:
tokenizer2 = BertTokenizer.from_pretrained('monsoon-nlp/bert-base-thai')
model2 = BertModel.from_pretrained("monsoon-nlp/bert-base-thai")

# This is the Model and Steps for the Training (Should be fine to run code sequentially starting here)

In [2]:
from transformers import EncoderDecoderModel, BertTokenizer,  EncoderDecoderConfig, BertConfig

In [3]:
import torch

In [34]:
import pandas as pd
import numpy as np
from nltk.translate import bleu
import tensorflow as tf

d = pd.read_csv("apdf.csv")

In [4]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

model = EncoderDecoderModel.from_encoder_decoder_pretrained('bert-base-multilingual-cased', 'bert-base-multilingual-cased')
model.train()

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertLMHeadModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertLMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertLMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertLMHeadModel were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['bert.encoder.layer.0.crossattention.self.query.weight', 'bert.encoder.layer.0.crossattention.self.query.bias', 'bert.encoder.layer.0.crossattention.self.key.weight', 'bert.encoder.layer.0.crossattention.self.key.bias

EncoderDecoderModel(
  (encoder): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_a

In [5]:
from transformers import AdamW
optimizer = AdamW(model.parameters(), lr=1e-5)

In [6]:
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]
optimizer = AdamW(optimizer_grouped_parameters, lr=1e-5)

In [26]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
text_batch = d["th_text"].head(2).tolist()
text_batchdec = d["en_text"].head(2).tolist()
encoding = tokenizer(text_batch, return_tensors='pt', padding=True, truncation=True)
input_ids = encoding['input_ids']
attention_mask = encoding['attention_mask']
decoding = tokenizer(text_batchdec, return_tensors='pt', padding=True, truncation=True)
decoder_ids = decoding['input_ids']
dec_attention_mask = decoding['attention_mask']

In [28]:
decoder_ids

tensor([[   101,  20794,  11273,  84977,  11565,  11090,    131,  47340,  10655,
          18051,  16730,  32764,  90233,  10219,  66868,    117,  10188,  12153,
            117,  28893,  10655,  18051,  16730,  21474,  15973,  10113,  16670,
          10111,  11596,  72413,  18051,  53966,  11220,  86090,  18332,  21333,
          15190,  26192,  28150,  11360,  10105,  36642, 105715,  10415,  20924,
          50986,  10106,  17779,  13998,  11584,    117,  36798,  10108,  32377,
            117,  20759,    117,  10106,  11120,  10302,    119,    164,  17443,
          17421,  10731,  49378,  35976,  29833,  11490,  23837,  76505,    166,
            102,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0],
        [   101,    113,  29582,  10162,    131,    138,  68415,  20718,  10114,
          10347,    169,   

In [27]:
decoder_ids2 = decoder_ids[:, 1:]

In [27]:
decoder_ids2.shape

torch.Size([2, 98])

In [28]:
zeros = torch.zeros((2, 1), dtype=torch.long)
zeros.shape

torch.Size([2, 1])

In [29]:
decoder_ids2 = torch.cat((decoder_ids2, zeros), 1)
decoder_ids2.shape

torch.Size([2, 99])

In [30]:
decoder_ids.shape

torch.Size([2, 99])

In [33]:
outputs = model(input_ids, attention_mask=attention_mask, decoder_input_ids=decoder_ids , labels=decoder_ids2, decoder_attention_mask=dec_attention_mask)
loss = outputs.loss
loss.backward()
optimizer.step()

In [32]:
outputs

Seq2SeqLMOutput(loss=tensor(12.2728, grad_fn=<NllLossBackward>), logits=tensor([[[-12.0510, -12.6997, -13.9370,  ..., -11.8948, -12.0700, -10.7809],
         [-12.3021, -11.7986, -13.0208,  ..., -13.1166, -12.6824, -11.6925],
         [-11.4336, -12.9474, -11.5324,  ..., -10.3358, -10.6908,  -8.2789],
         ...,
         [-10.7376,  -9.7483, -11.3937,  ..., -10.7604, -11.1043,  -9.5320],
         [-10.9605, -10.0709, -11.6784,  ..., -10.6306, -11.1322, -10.1532],
         [ -9.0608,  -8.3517,  -9.7376,  ...,  -8.8075,  -9.7575,  -8.2126]],

        [[-11.9221, -11.2450, -12.2737,  ..., -11.0527, -10.7908,  -9.9110],
         [-15.5917, -16.1420, -17.2587,  ..., -16.3634, -16.7450, -15.0727],
         [-13.6819, -13.0551, -15.3376,  ..., -15.8333, -16.6125, -14.0378],
         ...,
         [-12.2323, -11.5384, -13.1070,  ..., -11.8568, -13.2239, -11.9062],
         [-10.0410,  -9.6509, -11.4579,  ...,  -9.6324, -11.3007, -10.1906],
         [-12.9353, -12.2079, -14.1339,  ..., -12.9

In [ ]:
from transformers import Trainer, TrainingArguments, Seq2SeqTrainer

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total # of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
)

trainer = Seq2SeqTrainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=pd.DataFrame(tokenizer(d["th_text"].head(1000).tolist(), return_tensors='pt', padding=True, truncation=True), tokenizer(d["en_text"].head(1000).tolist(), return_tensors='pt', padding=True, truncation=True)),         # training dataset
    eval_dataset=pd.DataFrame(tokenizer(d["th_text"].head(1000).tolist(), return_tensors='pt', padding=True, truncation=True), tokenizer(d["en_text"].head(1000).tolist(), return_tensors='pt', padding=True, truncation=True)),         # training dataset,            # evaluation dataset
)

In [ ]:
trainer.train()

In [ ]:
tokenizer(d["en_text"].head(100).tolist())

In [ ]:
tokenizer.decode(input_ids[0])

In [ ]:
tokenizer.decode(decoder_ids[0])

In [ ]:
outputs

In [ ]:
lens = []
for i in d["th_text"].head(100):
    lens.append(len(i))
max_length = max(lens)

In [ ]:
def mt_model(max_input_length, train_layers, optimizer):
    """
    Implementation of NER model
    
    variables:
        max_input_length: number of tokens (max_length + 1)
        train_layers: number of layers to be retrained
        optimizer: optimizer to be used
    
    returns: model
    """
    
    tensors = tf.keras.layers.Input(shape=(max_length,))
    
    tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

    input_ids = tensors
    
    
    
    # Note: Bert layer from Hugging Face returns two values: sequence ouput, and pooled output. Here, we only want
    # the former. (See https://huggingface.co/transformers/model_doc/bert.html#tfbertmodel) 
    
    bert_layer = EncoderDecoderModel.from_encoder_decoder_pretrained('bert-base-multilingual-cased', 'bert-base-multilingual-cased')
    
    # Freeze layers, i.e. only train number of layers specified, starting from the top
    if not train_layers == -1:
        
        retrain_layers = []
    
        for retrain_layer_number in range(train_layers):

            layer_code = '_' + str(11 - retrain_layer_number)
            retrain_layers.append(layer_code)

        for w in bert_layer.weights:
            if not any([x in w.name for x in retrain_layers]):
                w._trainable = False

        # End of freezing section
    bert_sequence = bert_layer(input_ids=input_ids, decoder_input_ids=input_ids)[0]
    
    print('Let us check the shape of the BERT layer output:', bert_sequence)
    
    dense = tf.keras.layers.Dense(256, activation='relu', name='dense')(bert_sequence)
    
    dense = tf.keras.layers.Dropout(rate=0.1)(dense)
    
    pred = tf.keras.layers.Dense(21, activation='softmax', name='mt')(dense)
     
    print('pred: ', pred)
    
    ## Prepare for multipe loss functions, although not used here
    
    losses = {
        "mt": keras.losses.SparseCategoricalCrossentropy(),
        }
    lossWeights = {"mt": 1.0
                  }
    
    model = tf.keras.models.Model(inputs=bert_inputs, outputs=pred)

    model.compile(loss=losses, optimizer=optimizer, metrics=[bleu])
    
    
    model.summary()
    
    return model

In [ ]:
model = mt_model(max_length + 1, train_layers=-1, optimizer = tf.keras.optimizers.RMSprop)

model.fit(
    tokenizer(d["th_text"].head(100).tolist(), return_tensors='pt', padding=True), d["en_text"].head(100).tolist(), epochs = 10
)

In [ ]:
tokenizer(d["en_text"].head(100), return_tensors='tf', padding=True)

In [ ]:
d['en_text'].head(100).tolist()

In [ ]:
input_ids = torch.tensor(tokenizer.encode(testdata["th_text"][0], add_special_tokens=True)).unsqueeze(0)

In [ ]:
outputs = model(input_ids=input_ids, decoder_input_ids=input_ids)

In [ ]:
loss, logits = outputs.loss, outputs.logits

In [ ]:
generated = model.generate(input_ids, decoder_start_token_id=model.config.decoder.pad_token_id)

In [ ]:
tokenizer.decode(generated[0])

In [ ]:
from transformers import MT5Model, T5Tokenizer, TFMT5ForConditionalGeneration
import sentencepiece

In [ ]:
tokenizer = T5Tokenizer.from_pretrained("google/mt5-small")

In [ ]:
model = TFMT5ForConditionalGeneration.from_pretrained("google/mt5-small")

In [ ]:
inputs = tokenizer('translate English to German: The house is wonderful.', return_tensors='tf').input_ids
labels = tokenizer('Das Haus ist wunderbar.', return_tensors='tf').input_ids

In [ ]:
outputs = model(inputs, labels=labels)

In [ ]:
loss = outputs.loss
loss

In [ ]:
logits = outputs.logits
logits

In [ ]:
from transformers import T5Tokenizer, TFT5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained('t5-small')
model = TFT5ForConditionalGeneration.from_pretrained('t5-small')

In [ ]:
inputs = tokenizer("summarize: studies have shown that owning a dog is good for you ", return_tensors="tf").input_ids

In [ ]:
inputs = tokenizer("translate English to Spanish: studies have shown that owning a dog is good for you ", return_tensors="tf").input_ids

In [ ]:
result = model.generate(inputs)

In [ ]:
result

In [ ]:
tokenizer.decode(result[0])

# Work Using the GitHub Example (in Slack)

In [1]:
from transformers import Trainer, TrainingArguments, Seq2SeqTrainer, EncoderDecoderModel, BertTokenizer, EncoderDecoderConfig, BertConfig, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, default_data_collator
import pandas as pd
import numpy as np
from nltk.translate import bleu
import tensorflow as tf
import torch
from datasets import load_dataset, load_metric
from sklearn.model_selection import train_test_split

In [2]:
metric = load_metric("bleu")

In [3]:
# Use huggingface method to load
d = pd.read_csv("apdf.csv")

In [4]:
datasets = load_dataset('csv', data_files='apdf.csv')

Using custom data configuration default-45865a46c0640fbb
Reusing dataset csv (/home/jupyter/.cache/huggingface/datasets/csv/default-45865a46c0640fbb/0.0.0/2dc6629a9ff6b5697d82c25b73731dd440507a69cbce8b425db50b751e8fcfd0)


In [5]:
datasets

DatasetDict({
    train: Dataset({
        features: ['en_text', 'th_text'],
        num_rows: 13503
    })
})

In [6]:
train_dataset = datasets["train"]

In [7]:
train_dataset = train_dataset.select(range(100))
train_dataset

Dataset({
    features: ['en_text', 'th_text'],
    num_rows: 100
})

In [8]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

model = EncoderDecoderModel.from_encoder_decoder_pretrained('bert-base-multilingual-cased', 'bert-base-multilingual-cased')

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertLMHeadModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertLMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertLMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertLMHeadModel were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['bert.encoder.layer.0.crossattention.self.query.weight', 'bert.encoder.layer.0.crossattention.self.query.bias', 'bert.encoder.layer.0.crossattention.self.key.weight', 'bert.encoder.layer.0.crossattention.self.key.bias

In [9]:
# In Example, not sure if to keep -- we'll see if this works
model.config.decoder_start_token_id = tokenizer.convert_tokens_to_ids("thai")

In [10]:
print(model.config.decoder_start_token_id)

101379


In [11]:
column_names = datasets["train"].column_names

In [13]:
max_source_length = max([len(tokenizer(x, return_tensors='pt', truncation=True)["input_ids"][0]) for x in d['th_text']])
max_target_length = max([len(tokenizer(x, return_tensors='pt', truncation=True)["input_ids"][0]) for x in d['en_text']])

In [14]:
max_target_length

365

In [43]:
def preprocess_function(examples):
        inputs = [ex for ex in examples["th_text"]]
        targets = [ex for ex in examples["en_text"]]
        model_inputs = tokenizer(inputs, max_length=max_source_length, padding="max_length", truncation=True)

        # Setup the tokenizer for targets
        decoder = tokenizer(targets, return_tensors='pt',max_length=max_target_length, padding="max_length", truncation=True)
    
        decoder_ids = decoder["input_ids"]
        decoder_ids = decoder_ids[:, 1:]
        zeros = torch.zeros((len(examples["en_text"]), 1), dtype=torch.long)
        decoder_ids = torch.cat((decoder_ids, zeros), 1)

        # Was Label Previously
        model_inputs["decoder_input_ids"] = decoder["input_ids"].tolist()
        model_inputs["labels"] = decoder_ids.tolist()

        return model_inputs

In [44]:
train_dataset = train_dataset.map(
            preprocess_function,
            batched=True,
            remove_columns=column_names,
        )

In [45]:
# Replaced by above
#train_dataset = preprocess_function(datasets)

In [46]:
def postprocess_text(preds, labels):
        preds = [pred.strip() for pred in preds]
        labels = [[label.strip()] for label in labels]

        return preds, labels

In [52]:
def compute_metrics(eval_preds):
        preds, labels = eval_preds
        if isinstance(preds, tuple):
            preds = preds[0]
        decoded_preds = tokenizer.decode(preds, skip_special_tokens=True)
       
        decoded_labels = tokenizer.decode(labels, skip_special_tokens=True)

        # Some simple post-processing
        decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

        result = metric.compute(predictions=decoded_preds, references=decoded_labels)
        result = {"bleu": result["score"]}

        #prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
        #result["gen_len"] = np.mean(prediction_lens)
        #result = {k: round(v, 4) for k, v in result.items()}
        return result

In [53]:
data_collator = default_data_collator

In [54]:
training_args = Seq2SeqTrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total # of training epochs
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=8,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',
)

In [55]:
trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        #eval_dataset=eval_dataset,
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
    )

In [56]:
trainer.train()

Step,Training Loss


TrainOutput(global_step=39, training_loss=2.2174301147460938, metrics={'train_runtime': 1170.0236, 'train_samples_per_second': 0.033, 'total_flos': 606489928644600, 'epoch': 3.0})

In [57]:
trainer.metrics

AttributeError: 'Seq2SeqTrainer' object has no attribute 'metrics'